In [56]:
import requests
import re
import bs4
import pandas as pd
import csv
import time

headers = {
        'accept':r'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'user-agent':r'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
        'referer': r'https://www.qcc.com/',
        'accept-encoding':'gzip, deflate, br',
        'accept-language':'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
        'cookie':r'acw_tc=65e3181916080179256681092eaaa7bb43607bc41bfedcb5bcd5e4f74a; QCCSESSID=c6dghoqh88dkemfikaps6rgh85; UM_distinctid=176655658f817c-0fd68cdf6af0be-1e2e1b0b-1fa400-176655658f95fe; zg_did=%7B%22did%22%3A%20%221766556592e188-0c744b93544947-1e2e1b0b-1fa400-1766556592f639%22%7D; hasShow=1; _uab_collina=160801792663464258680485; CNZZDATA1254842228=1139550566-1608012683-https%253A%252F%252Fcn.bing.com%252F%7C1608017351; zg_de1d1a35bfa24ce29bbf2c7eb17e6c4f=%7B%22sid%22%3A%201608017926450%2C%22updated%22%3A%201608018962390%2C%22info%22%3A%201608017926453%2C%22superProperty%22%3A%20%22%7B%7D%22%2C%22platform%22%3A%20%22%7B%7D%22%2C%22utm%22%3A%20%22%7B%7D%22%2C%22referrerDomain%22%3A%20%22cn.bing.com%22%7D',
        }

with open("test.csv","w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['公司名字','公司标签','法定法人','注册资本','成立日期','法人邮箱','法人电话','公司地址','公司状态'])

    for count in range(500):
        if count == 0:
            url = 'https://www.qcc.com/g_HUN'
        else:
            url = 'https://www.qcc.com/g_HUN_' + str(count + 1) 
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                soup = bs4.BeautifulSoup(response.text, 'lxml')
                com_all_info = soup.find_all(class_='m_srchList')[0].tbody
                com_all_info_array = com_all_info.select('tr')
                for i in range(len(com_all_info_array)):
                    temp_g_name = com_all_info_array[i].select('td')[1].select('.ma_h1')[0].text    #获取公司名
                    temp_g_tag = com_all_info_array[i].select('td')[1].select('.search-tags')[0].text    #获取公司标签
                    temp_r_name = com_all_info_array[i].select('td')[1].select('p')[0].a.text    #获取法人名
                    temp_g_money = com_all_info_array[i].select('td')[1].select('p')[0].select('span')[0].text.strip('注册资本：')    #获取注册资本
                    temp_g_date = com_all_info_array[i].select('td')[1].select('p')[0].select('span')[1].text.strip('成立日期：')    #获取公司注册时间
                    temp_r_email = com_all_info_array[i].select('td')[1].select('p')[1].text.split('\n')[1].strip().strip('邮箱：')    #获取法人Email
                    temp_r_phone = com_all_info_array[i].select('td')[1].select('p')[1].select('.m-l')[0].text.strip('电话：')    #获取法人手机号
                    temp_g_addr = com_all_info_array[i].select('td')[1].select('p')[2].text.strip().strip('地址：')    #获取公司地址
                    temp_g_state = com_all_info_array[i].select('td')[2].select('span')[0].text  #获取公司状态
#                     dataframe = pd.DataFrame({'公司名字':temp_g_name,'公司标签':temp_g_tag,'法定法人':temp_r_name,'注册资本':temp_g_money,'成立日期':temp_g_date,'法人邮箱':temp_r_email,'法人电话':temp_r_phone,'公司地址':temp_g_addr,'公司状态':temp_g_state}, index=[0])
#                     dataframe.to_csv("test.csv", index=False, sep=',')
                    writer.writerow([temp_g_name, temp_g_tag, temp_r_name, temp_g_money, temp_g_date, temp_r_email, temp_r_phone, temp_g_addr, temp_g_state])
                    print(f'公司名:{temp_g_name}, 公司标签:{temp_g_tag}, 法人:{temp_r_name}, 电话:{temp_r_phone}, 状态:{temp_g_state}')           
            time.sleep(3)
        except requests.RequestException:
            print('except')

公司名:张国扬--安平工贸市场对面, 公司标签: , 法人:张国扬, 电话:-, 状态:存续
公司名:湖南高桥大市场有春电器店, 公司标签: , 法人:唐有春, 电话:-, 状态:注销
公司名:长沙市雨花区能言通讯, 公司标签: , 法人:陈展, 电话:-, 状态:注销
公司名:魏南--长沙市科星舞厅, 公司标签: , 法人:魏南, 电话:-, 状态:注销
公司名:陈辉锋, 公司标签: , 法人:陈辉锋, 电话:-, 状态:存续
公司名:衡阳市雁峰区祺祺副食经营部, 公司标签: , 法人:袁书成, 电话:-, 状态:注销
公司名:常宁市金鹏水果食品店, 公司标签: , 法人:尹新梅, 电话:-, 状态:存续
公司名:衡阳市雁峰区辉记排档, 公司标签: , 法人:蒋仲春, 电话:-, 状态:存续
公司名:韶山市永义亭移动通信代办点, 公司标签: , 法人:胡科, 电话:-, 状态:存续
公司名:陈士, 公司标签: , 法人:陈士, 电话:-, 状态:存续
公司名:浏阳市永和镇福源卤味店, 公司标签: , 法人:欧仁杰, 电话:-, 状态:存续
公司名:长沙市雨花区鑫迪汽车配件经营部, 公司标签: , 法人:曾全平, 电话:-, 状态:注销
公司名:汝城县柴火灶酒庄, 公司标签: , 法人:陈莲华, 电话:-, 状态:存续
公司名:长沙县安沙镇好靓迪歌厅, 公司标签: , 法人:车正红, 电话:-, 状态:存续
公司名:芦淞区明祥日用品商行, 公司标签: , 法人:罗明祥, 电话:-, 状态:存续
公司名:武冈市金雷定制衣柜专店, 公司标签: , 法人:毛建忠, 电话:-, 状态:存续
公司名:长沙市天心区雄安房产中介服务部, 公司标签: , 法人:陈雄, 电话:-, 状态:存续
公司名:安化县梅城镇鸿丰商行, 公司标签: , 法人:姚文才, 电话:-, 状态:存续
公司名:长沙县星沙槑得说零食店, 公司标签: , 法人:董凤玲, 电话:-, 状态:存续
公司名:炎陵县湘山建材配件店, 公司标签: , 法人:罗秀平, 电话:13786323598, 状态:存续
公司名:鹤城区曾明冻品经营部, 公司标签: , 法人:曾明, 电话:-, 状态:存续
公司名:麻阳老苗医养生堂, 公司标签: , 法人:谢桂花, 电话:-, 状态:存续
公司名:攸县琴缘足浴店, 公司标签: , 

AttributeError: 'NoneType' object has no attribute 'text'